# Classification of Ampules

First, make sure to select **GPU** as runtime type under Runtime > Change Runtime Type


In [6]:
!git clone https://github.com/ju-leon/ClassifyAmpules.git
%cd ClassifyAmpules

fatal: destination path 'ClassifyAmpules' already exists and is not an empty directory.
/content/ClassifyAmpules


Whenever you update your code remotely, just run the following command to refresh it:

In [30]:
!git fetch --all
!git reset --hard origin/master 

Fetching origin
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/ju-leon/ClassifyAmpules
   0a9fc55..77b00da  master     -> origin/master
HEAD is now at 77b00da Fix arguments


Next, we need to connect to our **Google Drive** to load and store our data:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Preprocessing

First, we need to scale down our data.

Repace the path with the path where your data is stored.

In [12]:
#Change these paths
in_path = "/content/drive/My Drive/Hackaton/PIM_Dataset"
cache_path = "/content/drive/My Drive/Hackaton/cache"
out_path = "/content/drive/My Drive/Hackaton/out"


#Do not touch
in_path_good = "'" + in_path + "/good_piece/" + "'"
in_path_bad = "'" + in_path + "/bad_piece/" + "'"

cache_path_good = "'" + cache_path + "/good_piece/" + "'"
cache_path_bad = "'" + cache_path + "/bad_piece/" + "'"

out_path = "'" + out_path + "/" + "'"


In [ ]:
!python preprocess.py $in_path_good $cache_path_good
!python preprocess.py $in_path_bad $cache_path_bad

# Combining

Now we combine 3 images into one.

In [ ]:
!python3 combine.py $cache_path_good $out_path_good
!python3 combine.py $cache_path_bad $out_path_bad

# Training

In [15]:
!python3 train.py $out_path

Using TensorFlow backend.
2020-07-02 20:27:01.730033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Found 7 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
(100, 100, 3)
/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
2020-07-02 20:27:04.191033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-02 20:27:04.214650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-02 20:27:04.215301: I tensorflow/core/common_runtime/gpu/gpu_device